In [8]:
import os
import json
import pandas as pd
import langchain
import langchain_groq
import traceback

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [74]:
from operator import itemgetter

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser




In [114]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(
    temperature=0,
    model="mixtral-8x7b-32768",
   
    api_key=os.getenv("GROQ_API_KEY"), # Optional if not set as an environment variable
    
)

system = "You are a witty assistant that likes giving relevant analogies to explain queries."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency for LLMs."})

AIMessage(content="Sure, I'd be happy to explain the importance of low latency for Large Language Models (LLMs) using an analogy!\n\nImagine you're in a conversation with a friend, and every time you ask a question, there's a delay of several seconds before they respond. At first, it might not seem like a big deal, but as the conversation goes on, those few seconds start to add up. It becomes frustrating to wait for a response, and you might even start to lose your train of thought.\n\nThe same concept applies to LLMs. Low latency is important because it allows the model to respond quickly and efficiently, providing a more seamless and natural interaction. If there's a delay in the model's response time, it can disrupt the flow of the conversation and make it more difficult for users to engage with the model.\n\nFurthermore, in certain applications, low latency is critical for ensuring a smooth user experience. For example, in a real-time chatbot or a voice assistant, users expect imme

In [14]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [212]:
print(json.dumps(RESPONSE_JSON))

{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}


In [206]:
TEMPLATE=r"""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like the json format given below  and use it as a guide.
Ensure to make {number} MCQs

{response_json}

The json keys and values MUST be in double quotes(""). Ensure once again that the json keys and values are encased in double quotes("")
also, i dont want any newline characters in the output. Ensure that there are no newline characters in the output

"""

In [193]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [194]:
quiz_chain= quiz_generation_prompt | chat 

In [217]:
from src.mcqgenerator.utils import get_table_data

In [218]:
AI_message= quiz_chain.invoke(dict_passed_in)
print(get_table_data(AI_message.content))

[{'MCQ': 'In what year was Ravikant born?', 'Choices': 'a-> 1974 || b-> 1984 || c-> 1994 || d-> 1964', 'Correct': 'a'}, {'MCQ': 'Which law firm did Ravikant intern at during college?', 'Choices': 'a-> Davis Polk & Wardwell || b-> Cravath, Swaine & Moore || c-> Sullivan & Cromwell || d-> Skadden, Arps, Slate, Meagher & Flom', 'Correct': 'a'}, {'MCQ': 'What is the name of the social media app that Ravikant co-founded in 2023?', 'Choices': 'a-> Airchat || b-> Chatly || c-> Socio || d-> Connect', 'Correct': 'a'}, {'MCQ': 'Which of the following companies did Hit Forge invest in?', 'Choices': 'a-> Twitter, Uber and Stack Overflow || b-> Google, Facebook and Amazon || c-> Microsoft, Apple and IBM || d-> Tesla, SpaceX and Neuralink', 'Correct': 'a'}, {'MCQ': "What is the name of the book that curates Ravikant's tweets, essays, and interviews on wealth and happiness?", 'Choices': "a-> The Almanack of Naval Ravikant || b-> Ravikant's Wealth Wisdom || c-> The Essential Naval || d-> Wealth and Ha

In [198]:
quiz_str= AI_message.content
quiz_str_stripped = quiz_str.strip()

In [177]:
def single_double_quote_converter(single_string):
    return single_string.replace("'", '"')

def get_table_data(quiz_str):
    try:
        
        # convert the quiz from a str to dict
        quiz_dict=json.loads(quiz_str, strict=False)
        quiz_table_data=[]
    
        
        # iterate over the quiz dictionary and extract the required information
        for key,value in quiz_dict.items():
            mcq=value["mcq"]
            options=" || ".join(
                [
                    f"{option}-> {option_value}" for option, option_value in value["options"].items()
                 
                 ]
            )
            
            correct=value["correct"]
            quiz_table_data.append({"MCQ": mcq,"Choices": options, "Correct": correct})
        
        return quiz_table_data
        
    except Exception as e:
        traceback.print_exception(type(e), e, e.__traceback__)
        return False


In [200]:

table_data= get_table_data(quiz_str=quiz_str)
print(type(table_data))


<class 'list'>


In [186]:
in_text= """Early life and education
Ravikant was born in New Delhi, India in 1974. He moved to New York with his mother and his brother, Kamal, when he was 9. He graduated from Stuyvesant High School in 1991.[6] In 1995, he graduated with degrees in Computer Science and Economics from Dartmouth College.[3] In college, he interned at law firm Davis Polk & Wardwell.[7] After graduating from Dartmouth College, Naval had a brief stint at Boston Consulting Group before heading to Silicon Valley.[7]

Career
Epinions
Main article: Epinions
In 1999, Ravikant co-founded consumer product review site Epinions.[8] They raised $45 million in venture capital from Benchmark Capital and August Capital.[8] In 2003, Epinions merged with comparison pricing site Dealtime with the approval of Ravikant and the other co-founders that had left the company—even though it meant valuing their shares at zero.[8]

The merged company became Shopping.com which held an IPO in October 2004.[8] After its first day of trading, it was worth $750 million.[8] In January 2005, Ravikant and three of his co-founders filed a lawsuit against Benchmark, August Capital, their co-founder Nirav Tolia who remained at Epinions after his co-founders' departures claiming that—to get their approval for the merger—they were misled to believe that at the time of the merger, the company was worth "$23 million to $38 million", less than the $45 million that they had raised in outside capital, making their shares worthless. The lawsuit was settled in December 2005.

Hit Forge
Around 2007, Ravikant started a $20 million early stage venture capital fund named "The Hit Forge".[9] Hit Forge invested in prominent startups including Twitter, Uber and Stack Overflow.[10][11][12]

AngelList
Main article: AngelList
In 2007, Ravikant began co-writing a blog called Venture Hacks, which "offered detailed advice on negotiating term sheets, explained which sections mattered, and which provisions were bogus."[13] That blog evolved into AngelList, which Ravikant co-founded in 2010, as a fundraising platform for startups to raise money from angel investors. AngelList also operates Product Hunt. In 2022, AngelList reached a $4 billion valuation.[14] Naval is the chairman and former CEO of AngelList.[2]

MetaStable Capital
In 2014, Ravikant co-founded MetaStable Capital, a cryptocurrency hedge fund that owned Bitcoin, Ethereum, Monero and a number of other cryptocurrencies. A June 2017 regulatory filing reported its assets as $69 million.[15] Investors in the fund include Andreessen Horowitz, Sequoia Capital, Union Square Ventures, Founders Fund and Bessemer Venture Partners.

Spearhead.co investment fund
In 2017, Naval launched Spearhead, an investment fund which raised $100m for its third fund to provide founders with $1m each to invest in technology companies as angel investors.[16] The first two classes of Spearhead include founders from 35 companies. Together, these companies are worth over $10B, and four of them are unicorns. The companies include Neuralink, Opendoor, PillPack, Shippo (company), Rippling and Scale.[17] Previous Spearhead leads include Shippo co-founder and chief executive officer Laura Behrens Wu, Scale AI founder and CEO Alexandr Wang and Rippling co-founder and chief technology officer Prasanna Sankar.[17]

Nav.al, Spearhead, and other podcasts
Naval runs a short-form podcast at Nav.al and Spearhead.co, where he discusses philosophy, business, and investing. He has also been a podcast guest on The Joe Rogan Experience, The Tim Ferriss Show, Coffee with Scott Adams, The James Altucher Show and Farnam Street, among others.

With Ravikant's permission, Eric Jorgenson curated Naval's tweets, essays, and interviews on wealth and happiness, then published them as a free downloadable book called The Almanack of Naval Ravikant, with a foreword by Tim Ferriss.[18]

Ravikant has a chapter giving advice in Tim Ferriss' book Tools of Titans.

Airchat
Ravikant co-founded Airchat in 2023, a social media app.[19] It uses Generative AI and is similar to Instagram stories.[19]"""

In [213]:
dict_passed_in={"text":in_text, "number":5 , "subject":"general people", "tone":"very hard", "response_json":json.dumps(RESPONSE_JSON)}

# Fiddling with generate + eval chain from here


In [83]:
quiz_chain_w_output_key= RunnablePassthrough.assign(quiz= quiz_chain)

In [89]:

response=quiz_chain_w_output_key.invoke(dict_passed_in)
print(response)

{'text': "Advanced topics in Newtonian physics at the undergraduate level typically include:\nLagrangian and Hamiltonian Formulations: These mathematical frameworks provide alternative ways to describe classical mechanics, emphasizing energy and momentum conservation. They are essential for understanding more complex systems and are often used in advanced courses.\nDeterministic Chaos Theory: This topic explores the intricate behavior of complex systems, such as the Poincaré surface-of-section, which helps students visualize and analyze the dynamics of chaotic systems.\nEuler's Rigid Body Equations of Motion: These equations describe the motion of rigid bodies, which is crucial for understanding many real-world phenomena, such as the motion of spacecraft and the behavior of molecules.\nCanonical Perturbation Theory: This method is used to study the long-term behavior of complex systems by analyzing small perturbations to their motion. It is essential for understanding many phenomena in

In [99]:
str_parser = StrOutputParser()
parsed_output = str_parser.parse(response.get("quiz"))



In [66]:
print(type(final_quiz))

<class 'langchain_core.messages.ai.AIMessage'>


In [35]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [49]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [87]:
full_prompt_chain= (
    quiz_chain_w_output_key
    |{
        "quiz":itemgetter("quiz"),
        "subject":itemgetter("subject")
    }
    | quiz_evaluation_prompt
    |chat
    |RunnablePassthrough.assign()
)

In [88]:
full_prompt_chain.invoke(dict_passed_in)

TypeError: RunnablePassthrough.assign() takes 1 positional argument but 2 were given